# A tutorial on how to use custom metrics for xgboost regressor

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


Use boston house prices data as an example.

In [2]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.Series(boston.target)


d:\Anaconda\envs\video\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e

In [3]:
X.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


<ul>CRIM per capita crime rate by town</ul>
<ul>ZN proportion of residential land zoned for lots over 25,000 sq.ft.</ul>
<ul>INDUS proportion of non-retail business acres per town</ul>
<ul>CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)</ul>
<ul>NOX nitric oxides concentration (parts per 10 million)</ul>
<ul>RM average number of rooms per dwelling</ul>
<ul>AGE proportion of owner-occupied units built prior to 1940</ul>
<ul>DIS weighted distances to five Boston employment centres</ul>
<ul>RAD index of accessibility to radial highways</ul>
<ul>TAX full-value property-tax rate per $10,000</ul>
<ul>PTRATIO pupil-teacher ratio by town</ul>
<ul>B 1000(Bk — 0.63)² where Bk is the proportion of blacks by town</ul>
<ul>LSTAT % lower status of the population</ul>
<ul>MEDV Median value of owner-occupied homes in $1000’s</ul>


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [5]:
regressor = xgb.XGBRegressor(
    n_estimators=102,
    reg_lambda=1,
    gamma=0,
    max_depth=3,
    grow_policy='lossguide',
)


In [6]:
regressor.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='lossguide',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=102, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [7]:
pd.DataFrame(regressor.feature_importances_.reshape(
    1, -1), columns=boston.feature_names)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.014932,0.011809,0.010745,0.007525,0.09699,0.351193,0.012626,0.058504,0.020932,0.029344,0.07024,0.020249,0.294911


In [8]:
y_pred = regressor.predict(X_test)


In [9]:
mean_squared_error(y_test, y_pred)


5.61589978572585

In [10]:
import torch
from torch.autograd import grad
import numpy as np
import xgboost as xgb
from typing import Tuple


d:\Anaconda\envs\video\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Our loss function will use the Squared Log Error (SLE):1/2[log(pred+1)-log(label+1)]^2


According to the Xgboost documentation, the loss function  need to return its gradient and Hessian matrix.
If the cost of calculate the Hessian matrix is too high, we can use the numpy.ones(gradient.shape) to replace the Hessian matrix.

In [11]:
# numpy version with gradient and hessian
#Manual calculate the gradient and hessian then implement the custom metric
def gradient(predt: np.ndarray, y) -> np.ndarray:
    # y = dtrain.get_label()
    res = (np.log(predt)-np.log1p(y))/(predt+1)
    return res


def hessian(predt: np.ndarray, y) -> np.ndarray:
    # y = dtrain.get_label()
    res = (-np.log1p(predt)+np.log1p(y)+1)/np.power(predt+1, 2)
    return res

#return the grad and hessian
def squared_log(predt: np.ndarray, y) -> np.ndarray:
    predt[predt < -1] = -1+1e-6
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess


In [12]:
# torch version with gradient and hessian
def squared_log_torch(predt: torch.Tensor, y: torch.Tensor):
    return 1/2 * (torch.log1p(predt) - torch.log1p(y))**2

#use torch autograd to simplify the calculation. No need to calculate the gradient and hessian 
def torch_autodiff_grad_hess(
    loss_function,
    y_true: np.ndarray, y_pred: np.ndarray
):

    y_true = torch.tensor(y_true, dtype=torch.float, requires_grad=False)#Don't need label gradient
    y_pred = torch.tensor(y_pred, dtype=torch.float, requires_grad=True)#requires_grad=True means that the gradient will be calculated
    loss_function_sum = lambda y_pred: loss_function(y_true, y_pred).sum()
    loss_function_sum(y_pred).backward()
    #Remember according to the pytorch autograd, grad can be implicitly created only for scalar outputs, we need to use sum here to let it backward, mean also works

    #gradient
    grad = y_pred.grad

    #hessian
    hess_matrix = torch.autograd.functional.hessian(loss_function_sum, y_pred, vectorize=True)
    hess = torch.diagonal(hess_matrix)

    return grad, hess


In [13]:
from functools import partial
torch_obj = partial(torch_autodiff_grad_hess, squared_log_torch)

In [14]:
regressor_custom = xgb.XGBRegressor(
    n_estimators=102,
    reg_lambda=1,
    gamma=0,
    max_depth=3,
    grow_policy='lossguide',
    objective=torch_obj
)


In [15]:
regressor_custom.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='lossguide',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=102, n_jobs=0,
             num_parallel_tree=1,
             objective=functools.partial(<function torch_autodiff_grad_hess at 0x0000016DD0F3B640>, <function squared_log_torch at 0x0000016DD0F3BD90>),
             predictor='auto', random_state=0, reg_alpha=0, ...)

In [16]:
y_pred = regressor_custom.predict(X_test)


In [17]:
mean_squared_error(y_test, y_pred)


89.67812492278979